## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [ ]:
class Huffman:
    @staticmethod
    def sort_list(lst):
        items = [(v, k) for k, v in lst.items()]
        items.sort(reverse=True)
        return {k: v for v, k in items}
    @staticmethod
    def create_table(data):
        list_of_nodes = []
        for i in set(data):
            list_of_nodes.append(Node(i, data.count(i)))
        list_of_nodes = sorted(list_of_nodes, key=lambda x: x.weight, reverse = True)
        temp1 = list_of_nodes.copy()
        list_of_trees = []
        list_of_trees.append(temp1.copy())
        while len(temp1) > 2:
            temp = []
            for i in range(len(temp1)-2):
                temp.append(temp1[i])
            rightnode = temp1[-2]
            leftnode = temp1[-1]
            temp.append(Node(leftnode.name + rightnode.name, leftnode.weight + rightnode.weight, None, leftnode, rightnode))
            temp1 = temp
            temp1 = sorted(temp1,  key=lambda x: x.weight, reverse = True)
            list_of_trees.append(sorted(temp1,  key=lambda x: x.weight, reverse = True))
        temp1[0].code = '1'
        temp1[1].code = '0'
        for i in list_of_trees[-2::-1]:
            # print(i)
            for nodee in i:
                if nodee.code == '':
                    if nodee.parent.left == nodee:
                        nodee.code += nodee.parent.code + '0'
                    else:
                        nodee.code += nodee.parent.code + '1'
                else:
                    continue
        leftnode = temp1[1]
        rightnode = temp1[0]
        return list_of_trees[0], Node(leftnode.name + rightnode.name, leftnode.weight + rightnode.weight, None, leftnode, rightnode)
    @staticmethod
    def encode(data):
        temp, root_node = Huffman.create_table(data)
        lst = dict()    
        table = dict()
        result = []
        lst = dict()
        for x in temp:
            lst[x.name] = x.weight
        items = [(v, k) for k, v in lst.items()]
        items.sort(reverse=True)
        list_of_chars = [k for v, k in items]
        for x in temp:
            table[x.name] = x.code
        # print(table)
        for char in data:
            result.append(table[char])
        return ''.join(result), root_node
    @staticmethod
    def decode(data, root_node):
        result = ''

        index = 0
        temp = ''
        node = root_node
        while index < len(data):
            node = root_node
            for i in data[index:]:
                if node.left != None:
                    if i == '0':
                        temp += i
                        node = node.left
                        index += 1
                    else:
                        temp += i
                        node = node.right
                        index += 1
                else:
                    result += node.name
                    # index += len(node.code)
                    break
            if index == len(data):
                result += node.name
                break
            print(index, len(data))
        return result

In [ ]:
#Test Huffman
data = 'I love my mother very much!'
encoded = Huffman.encode(data)
decoded = Huffman.decode(encoded)
print(decoded == data)

# Алгоритм LZW

In [ ]:
class LZW:
    def encode(self, data):
        dictionary = {chr(i):i for i in range(256)}
        next_val = 256
        temporary_for_check = ''
        compressed_data = []
        index = 0
        while index < len(data)-1:
            unit = data[index]
            if unit in dictionary.keys():
                temporary_for_check = unit
                for char_temp in data[index+1:]:
                    temporary_for_check += char_temp
                    if temporary_for_check in dictionary.keys():
                        continue
                    else:
                        index += len(temporary_for_check)-1
                        dictionary[temporary_for_check] = next_val
                        compressed_data.append(dictionary[temporary_for_check[:-1]])
                        next_val += 1
                        break
            if index == len(data) - 1:
                compressed_data.append(dictionary[temporary_for_check[-1]])
            temporary_for_check = ''
        return compressed_data
    def decompress(self, compressed_data):
        dictionary = {i:chr(i) for i in range(256)}
        next_value = 256
        previous_entry = dictionary[compressed_data[0]]
        decompressed_data = ''
        decompressed_data += previous_entry
        for i in compressed_data[1:]:
            if i in dictionary.keys():
                decompressed_data += dictionary[i]
                previous_entry += dictionary[i][0]
                if previous_entry not in dictionary.values():
                    dictionary[next_value] = previous_entry
                    next_value += 1
                previous_entry = dictionary[i]
            elif i == next_value:
                previous_entry += previous_entry[0]
                dictionary[next_value] = previous_entry
                decompressed_data += dictionary[next_value]

                previous_entry = dictionary[next_value]
                next_value += 1
        return decompressed_data


In [ ]:
lzw = LZW()
data = 'God bless America'
encoded = lzw.encode(data)
decoded = lzw.decode(encoded)
print(decoded == data)

# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [ ]:
class LZ77:
    def __init__(self, buffer_size: int):
        self.buff_size = buffer_size

    def encode(self, text: str) -> str:
        compressed_data = []
        bufsize = self.buff_size
        pos = 0
        max_length = len(data)
        while pos < len(data):
            dist_match = 0
            len_match = 0
            for i in range(1, min(max_length, len(data) - pos) + 1):
                substring = data[pos:pos + i]
                pre_occur = data[max(0, pos - bufsize): pos].rfind(substring)
                if pos - pre_occur <= bufsize and pre_occur != -1 :
                    len_match = i
                    dist_match = pos - pre_occur
                else:
                    break
            if len_match > 0:
                if pos + len_match < len(data):
                    next_char = data[pos + len_match] 
                compressed_data.append((dist_match, len_match, next_char))
                pos += len_match + 1
            else:
                compressed_data.append((0, 0, data[pos]))
                pos += 1
        return compressed_data

    def decode(self, code: str) -> str:
        '''
        Decoding for lz77
        '''
        decompressed_data = []
        for item in compressed_data:
            dist_match, len_match, next_char = item
            if len_match > 0:
                start_pos = len(decompressed_data) - dist_match
                for i in range(len_match):
                    decompressed_data.append(decompressed_data[start_pos + i])
            decompressed_data.append(next_char)
        return ''.join(decompressed_data)

In [ ]:
lz77 = LZ77(6)
data = 'I want my money back'
encoded = lz77.encode(data)
decoded = lz77.decode(encoded)
print(decoded == data)

# Алгоритм Deflate

In [ ]:
class Deflate:
    def __init__(self, buffer_size=6: int):
        self.buffer_size = buffer_size
    def encode(self, data):
        lz77 = LZ77(self.buffer_size)
        temp, node = Huffman.encode(data)
        return lz77.compress(temp), node
    def decode(self, data, node):
        lz77 = LZ77(self.buffer_size)
        return Huffman.decode(lz77.decompress(data), node)

In [ ]:
deflate = Deflate(6)
data = 'Long live Ukraine!'
encoded, node = deflate.encode(data)
decoded = deflate.decode(data)
print(decoded == data)